In [1]:
#!pip install keras

In [2]:
#!pip list keras

In [3]:
!python --version
#conda create -n "FYP2" python=3.6

#pip install -r requirements.txt

Python 3.6.13 :: Anaconda, Inc.


In [4]:
import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
# importing important libraries
from sklearn.feature_extraction.text import CountVectorizer 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [5]:
#importing libraries
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [6]:
#! cp C:/Users/Mr.Wick/Videos/Fall2020/FYP/utils.py
import sys  
sys.path.insert(0, 'C:/Users/Mr.Wick/Videos/Fall2020/FYP')

import utils

In [7]:
from utils import process_tweet, lookup  # For pre processing of tweets
import pdb
import math
import nltk
import string
import re   # Regular expression
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split

import csv

In [8]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [9]:
import xlrd
texts,labels = [], [] 
loc = ("S:/Spring2021/FYP-2/Dataset/Dataset1.xlsx")

wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)

for i in range(sheet.nrows):
    if i==0:
        continue
    texts.append(sheet.cell_value(i, 1))
    labels.append(sheet.cell_value(i,0))

In [10]:
# A function to pre process the tweets
def Pre_Process(Tweets):
    
    # remove stock market tickers like $GE

    #tweet = re.sub(r'\$\w*', '', tweet)

    # remove old style retweet text "RT"

     #  tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks

 #   tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags

    # only removing the hash # sign from the word

  #  tweet = re.sub(r'#', '', tweet)
    
    # tokenize tweets
    Tokenized_Tweets=[[]]
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True) # Case-Folding, Tokenization
    for i in range(len(Tweets)-1):
        tweet_tokens = tokenizer.tokenize(Tweets[i])
        Tokenized_Tweets.insert(i,tweet_tokens)
    #print("Number of Tokenized Tweets = "+str(len(Tokenized_Tweets)))
    #print(Tokenized_Tweets)
    #print("len(Tokenized)",len(Tokenized_Tweets))
    cleaned_tweets = []
    for i in range (len(Tokenized_Tweets)): # For all tweets
        inner=[]
        for k in range(0,len(Tokenized_Tweets[i])): # For all the words of one tweet
            check=0
            if Tokenized_Tweets[i][k] in string.punctuation:  # Removes punctuation
                check=1
            if not check:        
                inner.append(Tokenized_Tweets[i][k])
        cleaned_tweets.append(inner)
    return cleaned_tweets

In [11]:
cleaned_tweets=Pre_Process(texts)
print(cleaned_tweets)

[['karkhanon', 'ka', 'dhuwan', 'mahol', 'ko', 'aloodah', 'kerta', 'hai'], ['lahore', 'ke', 'mazeed', '7', 'elaaqay', 'seal', 'ker', 'diye', 'gaye'], ['jahaz', 'ke', 'pilots', 'aisey', 'bharti', 'kiye', 'gaye', 'jesay', 'jahaz', 'nahi', 'rickshaw', 'chalana', 'hai'], ['asif', 'ko', 'jhoot', 'bolney', 'ki', 'aadat', 'ne', 'sharmindah', 'kerwa', 'diya'], ['judge', 'khud', 'crime', 'karey', 'ga', 'to', 'usey', 'kon', 'judge', 'karey', 'ga'], ['eik', 'aue', 'hakumati', 'ittehadi', 'hakumat', 'se', 'naraz', 'ho', 'geya'], ['is', 'saal', 'hajj', 'sirf', 'muqami', 'log', 'adaa', 'ker', 'saken', 'gey'], ['mera', 'mashwarah', 'hai', 'ke', 'tum', 'aaj', 'ghar', 'hi', 'raho'], ['eik', 'dusrey', 'ke', 'khilaf', 'zeher', 'ugalney', 'se', 'behter', 'hai', 'ke', 'chup', 'raha', 'jaye'], ['puraman', 'talba', 'ko', 'giraftar', 'kerna', 'intehai', 'sharamnak', 'hai'], ['usey', 'maar', 'dena', 'chahiye', 'kyun', 'k', 'woh', 'hum', 'jins', 'parast', 'hai'], ['mulk', 'bhar', 'mein', 'jashan', 'e', 'azadi', 

In [12]:
trainDF = pandas.DataFrame()
trainDF['text'] = cleaned_tweets
trainDF['sentiment'] = labels
#data = data[['text','label']]
#print(type(data))
data=trainDF

In [13]:
data = data[data.sentiment != "Neutral"]

print(data[ data['sentiment'] == 0].size)
print(data[ data['sentiment'] == 1].size)


max_fatures = 6500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

5254
3740


In [14]:
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [15]:
# LSTM Model 
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation=tf.nn.sigmoid))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 49, 128)           832000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 49, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 1,087,194
Trainable params: 1,087,194
Non-trainable params: 0
_________________________________________________________________
None


In [16]:


Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, shuffle=True,stratify=Y)
#print(X_train.shape,Y_train.shape)
#print(Y_train[1])
#len(Y_train[:,1]) 
#print(Y_train[:,1])
#Y_train = Y_train.reshape( 3999, -1)
#print(X_test.shape,Y_test.shape)
print()


In [ ]:
history=model.fit(X_train, Y_train, epochs = 5, batch_size=64, verbose = 1)


In [ ]:
test_loss, test_acc = model.evaluate(X_test,Y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:

new_tweet = ['trump ne michelle obama ke barey mein nasal parastanah jumley kahey']
seq = tokenizer.texts_to_sequences(new_tweet)
padded = pad_sequences(seq, maxlen=150)
pred = model.predict(padded)
print(type(pred))
print(pred)
#print(pred, labels[np.argmax(pred)])
#predictions = model.predict(np.array([sample_text]))
#print(predictions)

In [ ]:
# save the model to disk
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('my_model.h5')

